# Hands on Machine Learning with SciKit Learn Keras and Tensorflow

## Chapter 7: Ensemble Methods and Random Forests

In this note book we'll be working through a dataset I found in Kaggle

### Dataset we'll be using: 

Placement Data Full Class by Ben Roshan

https://www.kaggle.com/benroshan/factors-affecting-campus-placement

We want to try and predict the "status" column, basically was someone placed or not

In [1]:
import pandas as pd

## Dataset Overview: 

### Overview
Placement data for students on a campus

### Columns
**sl_no**
* Serial Number

**gender**
* Gender M = Male | F = Female

**ssc_p**
* Secondary Education Percentage (10th Grade)

**ssc_b**
* Board of Education - Central/Others

**hsc_p**
* Higher Secondary Education Percentage - 12th Grade

**hsc_b**
* Board of Education - Central/Others

**hsc_s**
* Specialization in Higher Secondary Education

**degree_p**
* Degree Percentage

**degree_t**
* Undergrad Degree Type (Field of education)

**workex**
* Work Experience

**etest_p**
* Employability test percentage (conducted by college)

**specialisation**
* Post Grad (MBA) - Specialization

**mba_p**
* MBA percentage

**status**
* Status of placement - Placed/ Not placed

**salary**
* Salary offered to corporate candidates

In [2]:
data_import = pd.read_csv("Placement_Data_Full_Class.csv")
data_import

,sl_no,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,salary
0,1,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed,270000.0
1,2,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed,200000.0
2,3,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed,250000.0
3,4,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed,NaN
4,5,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed,425000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,211,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed,400000.0
211,212,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed,275000.0
212,213,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed,295000.0
213,214,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed,204000.0


In [3]:
data_import.describe() # No missing values (we don't care about salary)

,sl_no,ssc_p,hsc_p,degree_p,etest_p,mba_p,salary
count,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,148.000000
mean,108.000000,67.303395,66.333163,66.370186,72.100558,62.278186,288655.405405
std,62.209324,10.827205,10.897509,7.358743,13.275956,5.833385,93457.452420
min,1.000000,40.890000,37.000000,50.000000,50.000000,51.210000,200000.000000
25%,54.500000,60.600000,60.900000,61.000000,60.000000,57.945000,240000.000000
50%,108.000000,67.000000,65.000000,66.000000,71.000000,62.000000,265000.000000
75%,161.500000,75.700000,73.000000,72.000000,83.500000,66.255000,300000.000000
max,215.000000,89.400000,97.700000,91.000000,98.000000,77.890000,940000.000000


In [4]:
# Checking Data Types

for column in data_import.columns:
    print(column, data_import[column].dtype, len(data_import[column].unique()))

sl_no int64 215
gender object 2
ssc_p float64 103
ssc_b object 2
hsc_p float64 97
hsc_b object 2
hsc_s object 3
degree_p float64 89
degree_t object 3
workex object 2
etest_p float64 100
specialisation object 2
mba_p float64 205
status object 2
salary float64 46


## Data Cleansing:

As per usual, we'll need to clean our data before we can use it in our Machine Learning algorithms

1. Drop the sl_no column as we can't predict with it and drop salary as we're not going to use it
2. Label Encode: "Gender", "workex", "ssc_b", "hsc_b", "specialization"
3. One Hot Encode: "hsc_s", "degree_t"



In [5]:
dropped_columns = data_import.copy()

dropped_columns = dropped_columns.drop(["sl_no", "salary"], axis=1)
dropped_columns

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status
0,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed
1,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed
2,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed
3,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed
4,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed
211,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed
212,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed
213,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed


In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

label_encoded_data = dropped_columns.copy()

columns_to_label_encode = ["gender", "workex", "ssc_b", "hsc_b", "specialisation"]

label_encoded_data[columns_to_label_encode] = label_encoded_data[columns_to_label_encode].apply(le.fit_transform)

label_encoded_data

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status
0,1,67.00,1,91.00,1,Commerce,58.00,Sci&Tech,0,55.0,1,58.80,Placed
1,1,79.33,0,78.33,1,Science,77.48,Sci&Tech,1,86.5,0,66.28,Placed
2,1,65.00,0,68.00,0,Arts,64.00,Comm&Mgmt,0,75.0,0,57.80,Placed
3,1,56.00,0,52.00,0,Science,52.00,Sci&Tech,0,66.0,1,59.43,Not Placed
4,1,85.80,0,73.60,0,Commerce,73.30,Comm&Mgmt,0,96.8,0,55.50,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,1,82.00,1,Commerce,77.60,Comm&Mgmt,0,91.0,0,74.49,Placed
211,1,58.00,1,60.00,1,Science,72.00,Sci&Tech,0,74.0,0,53.62,Placed
212,1,67.00,1,67.00,1,Commerce,73.00,Comm&Mgmt,1,59.0,0,69.72,Placed
213,0,74.00,1,66.00,1,Commerce,58.00,Comm&Mgmt,0,70.0,1,60.23,Placed


In [7]:
hot_encoded_data = label_encoded_data.copy()

hot_encoded_data_y_placeholder = hot_encoded_data["status"]
hot_encoded_data = hot_encoded_data.drop("status", axis=1) # Removes the prediction column so that we don't encode it

hot_encoded_data = pd.get_dummies(hot_encoded_data)

hot_encoded_data = pd.concat([hot_encoded_data, hot_encoded_data_y_placeholder], axis=1)
hot_encoded_data

,gender,ssc_p,ssc_b,hsc_p,hsc_b,degree_p,workex,etest_p,specialisation,mba_p,hsc_s_Arts,hsc_s_Commerce,hsc_s_Science,degree_t_Comm&Mgmt,degree_t_Others,degree_t_Sci&Tech,status
0,1,67.00,1,91.00,1,58.00,0,55.0,1,58.80,0,1,0,0,0,1,Placed
1,1,79.33,0,78.33,1,77.48,1,86.5,0,66.28,0,0,1,0,0,1,Placed
2,1,65.00,0,68.00,0,64.00,0,75.0,0,57.80,1,0,0,1,0,0,Placed
3,1,56.00,0,52.00,0,52.00,0,66.0,1,59.43,0,0,1,0,0,1,Not Placed
4,1,85.80,0,73.60,0,73.30,0,96.8,0,55.50,0,1,0,1,0,0,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,1,82.00,1,77.60,0,91.0,0,74.49,0,1,0,1,0,0,Placed
211,1,58.00,1,60.00,1,72.00,0,74.0,0,53.62,0,0,1,0,0,1,Placed
212,1,67.00,1,67.00,1,73.00,1,59.0,0,69.72,0,1,0,1,0,0,Placed
213,0,74.00,1,66.00,1,58.00,0,70.0,1,60.23,0,1,0,1,0,0,Placed


In [8]:
# Scaling our data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_data = hot_encoded_data.copy()

scaled_data_temp = scaled_data.iloc[:, -1]
scaled_data = scaled_data.iloc[:, :-1]

scaled_data = scaler.fit_transform(scaled_data)

# scaled_data = pd.concat([scaled_data, scaled_data_temp], axis=1)
scaled_data

array([[ 0.73943397, -0.02808697,  1.08245885, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [ 0.73943397,  1.11336869, -0.92382264, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [ 0.73943397, -0.21323793, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384],
       ...,
       [ 0.73943397, -0.02808697,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [-1.35238581,  0.61994138,  1.08245885, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [ 0.73943397, -0.49096436, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384]])

## Train Test Split

In [9]:
from sklearn.model_selection import train_test_split

X = scaled_data
y = scaled_data_temp


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train

array([[ 0.73943397, -0.39838889,  1.08245885, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [-1.35238581,  0.20335173, -0.92382264, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [-1.35238581, -0.7686908 , -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384],
       ...,
       [ 0.73943397,  1.36054521,  1.08245885, ..., -1.43924583,
        -0.23221018,  1.62605898],
       [ 0.73943397, -0.25026812, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384],
       [ 0.73943397, -1.41671915, -0.92382264, ...,  0.69480833,
        -0.23221018, -0.61498384]])

## Run ML Algorithms

### Build Ensemble Predictor

In [10]:
# Instantiating and configuring our Ensemble Classifier

from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_for_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(estimators = [('lr', log_clf), ('rf', rnd_for_clf), ('sc', svm_clf)], voting="hard")


In [11]:
# Training the Ensemble classifier

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('sc', SVC())])

## Measure the accuracy of our classifier

In [12]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_for_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.8732394366197183
RandomForestClassifier 0.8450704225352113
SVC 0.8028169014084507
VotingClassifier 0.8732394366197183


## Bagging and Pasting in skLearn

In [13]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1) # Bootstrap determines whether this is bagging or pasting
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)


## Random Forests

In [14]:
## Create a random forest classifier and see the importance of each variable

from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rf_clf.fit(X_train, y_train)

for name, score in zip(hot_encoded_data.iloc[:, :-1].columns, rf_clf.feature_importances_):
    print(name, ": ", score)


gender :  0.021213645969012426
ssc_p :  0.285979806716795
ssc_b :  0.020802299442869737
hsc_p :  0.19419952451734918
hsc_b :  0.014446449812059576
degree_p :  0.17782372191803192
workex :  0.042369015290948664
etest_p :  0.06005267062805941
specialisation :  0.027515104371099025
mba_p :  0.11128298632302067
hsc_s_Arts :  0.005327580885848548
hsc_s_Commerce :  0.008268399427410025
hsc_s_Science :  0.006734929256143214
degree_t_Comm&Mgmt :  0.008833602489949055
degree_t_Others :  0.005364964689293418
degree_t_Sci&Tech :  0.009785298262110192


### AdaBoost

In [15]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200, 
    algorithm="SAMME.R", learning_rate=0.5)

ada_clf.fit(X_train, y_train)

y_pred = ada_clf.predict(X_test)

accuracy_score(y_test, y_pred)

0.8309859154929577

### Gradient Boosting

In [16]:
from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(max_depth=2, n_estimators=120)

gbrt.fit(X_train, y_train)

y_pred = gbrt.predict(X_test)

accuracy_score(y_test, y_pred)

0.8450704225352113

In [17]:
import xgboost

ModuleNotFoundError: No module named 'xgboost'